In [15]:
import import_ipynb
from LoadPicture import *

import numpy as np
import glob

# Tensorflow imports
from tensorflow import keras
from tensorflow.keras.applications.resnet50 import ResNet50V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

## Model parameters

In [16]:
batch_size = 32

## Model callbacks

In [17]:
callbacks = [
    keras.callbacks.ModelCheckpoint(                        
                        filepath='../log/cps/{model.name}_cp_{epoch}', 
                        verbose=1, 
                        save_weights_only=False,
                        save_freq=5*batch_size) # Checkpoint is saved every five epochs
]

## Data parameters

In [18]:
image_height = 178
image_width = 218

## Creating the training dataset

In [ ]:
training_ds = []

## Creating the validation dataset

In [ ]:
validation_ds = []

## Creating the test dataset

In [ ]:
test_ds = []

## Load the base model

In [19]:
# Loading either the ResNet50 architecture model or the previously saved model, and freeze it for transfer learning
base_model = ResNet50V2(include_top=False, classes=2)
base_model.trainable = False

## Creating a new model

In [20]:
inputs = keras.Input(shape=(image_height, image_width, 3))

# Running base model in inference mode
x = base_model(inputs, training=False)

# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)

# Final layer for binary classification
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

## Training the new model with new data

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])
model.fit(training_ds, epochs=20, callbacks=callbacks, validation_data=...)